In [25]:
# system
import os
import re
import sys
from glob import glob


# data preprocessing
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

# EEG data preprocessing
import mne
from mne.event import define_target_events
from mne.channels import make_1020_channel_selections
from mne.preprocessing import (ICA, create_eog_epochs, create_ecg_epochs, corrmap)

# ml
import sklearn

# statistics 
import scipy.stats as stats
import statsmodels.api as sm
from scipy.stats import ttest_1samp
from statsmodels.stats.anova import AnovaRM

In [26]:
print("Python version is", sys.version)
print("MNE version is", mne.__version__)

Python version is 3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]
MNE version is 1.2.1


# Extract Behavior(tsv) data from BIDS folder and merge them 

df_dirs: path + subj_idx + task + datatype + run

**e.g.** './BIDS_data/sub-003/sourcedata-eeg_inside-MRT/beh/sub-003_task-pdm_acq-insideMRT_run-01_beh.csv'

data in outside/inside-MRT is split by 'run' times.

data in outside-MRT is **not** split by 'run' times.

data in inside-MRT is split by 'run' times.

the load pipeline doesn't consider fMRI
.

path name

**e.g.** './BIDS_data'

In [27]:
# cur_dir:./Projects/[projectname]/2_study/2_4_Analysis/2_4_2_Scripts[workingdir]'
temp_dir =os.path.dirname(               # ./
        os.path.dirname(                 # /3_projects
            os.path.dirname(             # /[projectname]
                os.path.dirname(         # /2_study
                    os.path.dirname(     # /2_4_Analysis
                        os.getcwd()      # /2_4_2_Scripts(workingdir)
                        )))))

# data_dir:./'2_Data/Ostwald2018/BIDS_data/...'
typename = 'Data'
dataname = 'Ostwald2018'
BIDSname = 'BIDS_data' 
datadir = os.path.join(typename, dataname, BIDSname)
datadir = os.path.normpath(datadir)

subj_idx name

**e.g.** 'sub-001'

In [28]:
# subj_idx
subjects = ['sub-001', 
                 #'sub-002', # there is no outside beh or eeg data
                 'sub-003', 
                 'sub-004', 
                 'sub-005', 
                 'sub-006', 
                 'sub-007', 
                 'sub-008', 
                 'sub-009', 
                 'sub-010', 
                 'sub-011', 
                 'sub-012',
                 'sub-013', 
                 'sub-014', 
                 'sub-015', 
                 'sub-016',
                 'sub-017']

task name

**e.g.** 'sourcedata-eeg_outside-MRT'


In [29]:
tasks = {
        'outside':'sourcedata-eeg_outside-MRT',
        'inside':'sourcedata-eeg_inside-MRT',
        'fmri':'sourcedata-fmri'
    }

datatype name

**e.g.** 'beh'


In [30]:
datatypes = {'beh':'beh','eeg':'eeg'}

run name

**e.g.** 'outsideMRT_run-01_beh.tsv'


In [31]:
# runs
outside_runs=[]
inside_runs=[]

# outside MRI only has 2 runs
side = 'outside'
for i in range(2):
    run = '_task-pdm_acq-'+side+'MRT_run-0'+str(i+1)+'_beh.tsv'
    outside_runs.append(run)

# inside MRI has 5 runs
side = 'inside'    
for i in range(5):
    run = '_task-pdm_acq-'+side+'MRT_run-0'+str(i+1)+'_beh.tsv'
    inside_runs.append(run)

# zip into a dict
runs = {
        'outside':outside_runs,
        'inside':inside_runs
}
runs

{'outside': ['_task-pdm_acq-outsideMRT_run-01_beh.tsv',
  '_task-pdm_acq-outsideMRT_run-02_beh.tsv'],
 'inside': ['_task-pdm_acq-insideMRT_run-01_beh.tsv',
  '_task-pdm_acq-insideMRT_run-02_beh.tsv',
  '_task-pdm_acq-insideMRT_run-03_beh.tsv',
  '_task-pdm_acq-insideMRT_run-04_beh.tsv',
  '_task-pdm_acq-insideMRT_run-05_beh.tsv']}

In [32]:
# side and datatype
side = 'outside'
datatype = 'beh'

In [33]:
# list to load filename
df_dirs=[]


# subj_idx(list)
for subj_idx in subjects: 
# task(dict)
    task=tasks[side]
# datatype(dict)
    datatype=datatypes[datatype]
# run(dict)
    for run in runs[side]:
        run=subj_idx+run
# combine path
        df_dir=os.path.join(temp_dir, datadir, subj_idx, task, datatype, run)
        df_dir=os.path.normpath(df_dir)
# memory path
        df_dirs.append(df_dir)

Load data and record dir which has no file.

In [34]:
# df where to record dir which has file. 
df_has_dirs = []

# df where to record dir which has no file.
df_no_dirs = []


In [41]:
len(df)

144

In [36]:

# load data.
for df_dir in df_dirs:
    if os.path.exists(df_dir):
        df_has_dirs.append(df_dir)
        df = pd.read_csv(df_dir,sep='\t')
    else:
        df_no_dirs.append(df_dir)


In [37]:
df_no_dirs

[]

# Extract EEG(vhdr) data from BIDS folder and merge them 

df_dirs: path + subj_idx + task + datatype + run

**e.g.** './BIDS_data/sub-003/sourcedata-eeg_inside-MRT/beh/sub-003_task-pdm_acq-insideMRT_run-01_eeg.vhdr'

data in outside/inside-MRT is split by 'run' times.

data in outside-MRT is **not** split by 'run' times.

data in inside-MRT is split by 'run' times.

the load pipeline doesn't consider fMRI

path name

**e.g.** './BIDS_data'

In [13]:
# cur_dir:./Projects/[projectname]/2_study/2_4_Analysis/2_4_2_Scripts[workingdir]'
temp_dir =os.path.dirname(               # ./
        os.path.dirname(                 # /3_projects
            os.path.dirname(             # /[projectname]
                os.path.dirname(         # /2_study
                    os.path.dirname(     # /2_4_Analysis
                        os.getcwd()      # /2_4_2_Scripts(workingdir)
                        )))))

# data_dir:./'2_Data/Ostwald2018/BIDS_data/...'
typename = 'Data'
dataname = 'Ostwald2018'
BIDSname = 'BIDS_data' 
datadir = os.path.join(typename, dataname, BIDSname)
datadir = os.path.normpath(datadir)

subj_idx name

**e.g.** 'sub-001'

In [14]:
# subj_idx
subjects = ['sub-001', 
                 #'sub-002', # there is no outside beh or eeg data
                 'sub-003', 
                 'sub-004', 
                 'sub-005', 
                 'sub-006', 
                 'sub-007', 
                 'sub-008', 
                 'sub-009', 
                 'sub-010', 
                 'sub-011', 
                 'sub-012',
                 'sub-013', 
                 'sub-014', 
                 'sub-015', 
                 'sub-016',
                 'sub-017']

task name

**e.g.** 'sourcedata-eeg_outside-MRT'


In [15]:
tasks = {
        'outside':'sourcedata-eeg_outside-MRT',
        'inside':'sourcedata-eeg_inside-MRT',
        'fmri':'sourcedata-fmri'
    }

datatype name

**e.g.** 'beh'


In [16]:
datatypes = {'beh':'beh','eeg':'eeg'}

run name

**e.g.** 'outsideMRT_run-01_eeg.vhdr'


In [17]:
# runs
outside_runs=[]
inside_runs=[]

# outside MRI only has 2 runs, and they are in the same file
side = 'outside'
for i in range(1):
    run = '_task-pdm_acq-'+side+'MRT_eeg.vhdr'
    outside_runs.append(run)

# inside MRI has 5 runs
side = 'inside'    
for i in range(5):
    run = '_task-pdm_acq-'+side+'MRT_run-0'+str(i+1)+'_eeg.vhdr'
    inside_runs.append(run)

# zip into a dict
runs = {
        'outside':outside_runs,
        'inside':inside_runs
}
runs

{'outside': ['_task-pdm_acq-outsideMRT_eeg.vhdr'],
 'inside': ['_task-pdm_acq-insideMRT_run-01_eeg.vhdr',
  '_task-pdm_acq-insideMRT_run-02_eeg.vhdr',
  '_task-pdm_acq-insideMRT_run-03_eeg.vhdr',
  '_task-pdm_acq-insideMRT_run-04_eeg.vhdr',
  '_task-pdm_acq-insideMRT_run-05_eeg.vhdr']}

In [18]:
# side and datatype
side = 'outside'
datatype = 'eeg'

In [19]:
# list to load filename
df_dirs=[]


# subj_idx(list)
for subj_idx in subjects: 
# task(dict)
    task=tasks[side]
# datatype(dict)
    datatype=datatypes[datatype]
# run(dict)
    for run in runs[side]:
        run=subj_idx+run
# combine path
        df_dir=os.path.join(temp_dir, datadir, subj_idx, task, datatype, run)
        df_dir=os.path.normpath(df_dir)
# memory path
        df_dirs.append(df_dir)

In [20]:
# df where to record dir which has file. 
df_has_dirs = []

# df where to record dir which has no file.
df_no_dirs = []

# load data.
for df_dir in df_dirs:
    if os.path.exists(df_dir):
        df_has_dirs.append(df_dir)
        raw = mne.io.read_raw_brainvision(df_dir)
    else:
        df_no_dirs.append(df_dir)

Extracting parameters from f:\Data\Ostwald2018\BIDS_data\sub-001\sourcedata-eeg_outside-MRT\eeg\sub-001_task-pdm_acq-outsideMRT_eeg.vhdr...
Setting channel info structure...


C:\Users\liuyikang\AppData\Local\Temp\ipykernel_24560\938834549.py:11: RuntimeWarning: No coordinate information found for channels ['EOG', 'ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(df_dir)
C:\Users\liuyikang\AppData\Local\Temp\ipykernel_24560\938834549.py:11: RuntimeWarning: Not setting positions of 2 misc channels found in montage:
['EOG', 'ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne.io.read_raw_brainvision(df_dir)


Extracting parameters from f:\Data\Ostwald2018\BIDS_data\sub-003\sourcedata-eeg_outside-MRT\eeg\sub-003_task-pdm_acq-outsideMRT_eeg.vhdr...
Setting channel info structure...
Extracting parameters from f:\Data\Ostwald2018\BIDS_data\sub-004\sourcedata-eeg_outside-MRT\eeg\sub-004_task-pdm_acq-outsideMRT_eeg.vhdr...
Setting channel info structure...


C:\Users\liuyikang\AppData\Local\Temp\ipykernel_24560\938834549.py:11: RuntimeWarning: No coordinate information found for channels ['EOG', 'ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(df_dir)
C:\Users\liuyikang\AppData\Local\Temp\ipykernel_24560\938834549.py:11: RuntimeWarning: Not setting positions of 2 misc channels found in montage:
['EOG', 'ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne.io.read_raw_brainvision(df_dir)
C:\Users\liuyikang\AppData\Local\Temp\ipykernel_24560\938834549.py:11: RuntimeWarning: No coordinate information found for channels ['EOG', 'ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(df_dir)
C:\Users\liuyikang\AppData\Local\Temp\ipykernel_24560\93883454

Extracting parameters from f:\Data\Ostwald2018\BIDS_data\sub-005\sourcedata-eeg_outside-MRT\eeg\sub-005_task-pdm_acq-outsideMRT_eeg.vhdr...
Setting channel info structure...
Extracting parameters from f:\Data\Ostwald2018\BIDS_data\sub-006\sourcedata-eeg_outside-MRT\eeg\sub-006_task-pdm_acq-outsideMRT_eeg.vhdr...
Setting channel info structure...


...

C:\Users\liuyikang\AppData\Local\Temp\ipykernel_24560\938834549.py:11: RuntimeWarning: No coordinate information found for channels ['EOG', 'ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(df_dir)
C:\Users\liuyikang\AppData\Local\Temp\ipykernel_24560\938834549.py:11: RuntimeWarning: Not setting positions of 2 misc channels found in montage:
['EOG', 'ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne.io.read_raw_brainvision(df_dir)
C:\Users\liuyikang\AppData\Local\Temp\ipykernel_24560\938834549.py:11: RuntimeWarning: No coordinate information found for channels ['EOG', 'ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(df_dir)
C:\Users\liuyikang\AppData\Local\Temp\ipykernel_24560\93883454

Extracting parameters from f:\Data\Ostwald2018\BIDS_data\sub-014\sourcedata-eeg_outside-MRT\eeg\sub-014_task-pdm_acq-outsideMRT_eeg.vhdr...
Setting channel info structure...
Extracting parameters from f:\Data\Ostwald2018\BIDS_data\sub-015\sourcedata-eeg_outside-MRT\eeg\sub-015_task-pdm_acq-outsideMRT_eeg.vhdr...
Setting channel info structure...


C:\Users\liuyikang\AppData\Local\Temp\ipykernel_24560\938834549.py:11: RuntimeWarning: No coordinate information found for channels ['EOG', 'ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(df_dir)
C:\Users\liuyikang\AppData\Local\Temp\ipykernel_24560\938834549.py:11: RuntimeWarning: Not setting positions of 2 misc channels found in montage:
['EOG', 'ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne.io.read_raw_brainvision(df_dir)
C:\Users\liuyikang\AppData\Local\Temp\ipykernel_24560\938834549.py:11: RuntimeWarning: No coordinate information found for channels ['EOG', 'ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(df_dir)
C:\Users\liuyikang\AppData\Local\Temp\ipykernel_24560\93883454

Extracting parameters from f:\Data\Ostwald2018\BIDS_data\sub-016\sourcedata-eeg_outside-MRT\eeg\sub-016_task-pdm_acq-outsideMRT_eeg.vhdr...
Setting channel info structure...
Extracting parameters from f:\Data\Ostwald2018\BIDS_data\sub-017\sourcedata-eeg_outside-MRT\eeg\sub-017_task-pdm_acq-outsideMRT_eeg.vhdr...
Setting channel info structure...


C:\Users\liuyikang\AppData\Local\Temp\ipykernel_24560\938834549.py:11: RuntimeWarning: No coordinate information found for channels ['EOG', 'ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(df_dir)
C:\Users\liuyikang\AppData\Local\Temp\ipykernel_24560\938834549.py:11: RuntimeWarning: Not setting positions of 2 misc channels found in montage:
['EOG', 'ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne.io.read_raw_brainvision(df_dir)
C:\Users\liuyikang\AppData\Local\Temp\ipykernel_24560\938834549.py:11: RuntimeWarning: No coordinate information found for channels ['EOG', 'ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(df_dir)
C:\Users\liuyikang\AppData\Local\Temp\ipykernel_24560\93883454

In [21]:
print(df_no_dirs)

[]
